In [114]:
pip install langchain langchain_community langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [115]:
import os
import getpass

os.environ["LANGCHAIN_V2_TRACING"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com/"
os.environ["LANGCHAIN_PROJECT"] = "rag-app" 
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Langchain API Key is: ")

Langchain API Key is: ········


In [3]:
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

········


In [116]:
pip install openai langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [66]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key is: ")

OpenAI API Key is: ········


In [117]:
from langchain.callbacks import LangChainTracer, tracing_v2_enabled
from langchain.callbacks.manager import CallbackManager

In [118]:
pip install -qU langchain-anthropic

Note: you may need to restart the kernel to use updated packages.


In [55]:
from langchain_anthropic import ChatAnthropic

tracer = LangChainTracer(project_name="langchain-rag-app")
callback_manager = CallbackManager([tracer])


llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", callback_manager = callback_manager)

In [56]:
with tracing_v2_enabled():
    response = llm.invoke("Is capitalism better than socialism/communism. Give me a straight answer, dont be diplomatic")
print(response.content)

Based on historical evidence and economic outcomes, capitalism has generally proven to be more effective than socialism or communism at generating economic growth, innovation, and improving overall living standards. Capitalist economies have typically outperformed socialist or communist ones in terms of productivity, technological advancement, and providing a higher quality of life for their citizens. However, this is a complex and controversial topic, and there are valid criticisms of both systems.


In [119]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

In [138]:
print(docs[0].page_content[:30000])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

In [121]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

66

In [122]:
all_splits[1]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 971}, page_content='Memory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.')

In [123]:
len(all_splits[0].page_content)

969

In [124]:
all_splits[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 8}

In [125]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [126]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("LLM's function as what in a LLM-powered autonomous agent system")

len(retrieved_docs)

6

In [127]:
print(retrieved_docs[0].page_content)

LLM Powered Autonomous Agents


In [128]:

from langchain_anthropic import ChatAnthropic

tracer = LangChainTracer(project_name="langchain-rag-app")
callback_manager = CallbackManager([tracer])

llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0.2, max_tokens=1024, callback_manager = callback_manager)



In [129]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [130]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [135]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

with tracing_v2_enabled():
    response = rag_chain.invoke("What is LLM's function in LLM-powered autonomous agent system")
print(response)

In an LLM-powered autonomous agent system, the LLM functions as the core controller or "brain" of the agent. Its main roles include:

1) Planning and task decomposition: Breaking down complex tasks into smaller, manageable subgoals.

2) Reflection and refinement: Performing self-criticism, learning from mistakes, and refining actions for better results.

3) Utilizing external tools and memory components to augment its capabilities.


In [139]:
with tracing_v2_enabled():
    response1 = rag_chain.invoke("How does huggingface work?")
print(response1)

HuggingGPT is a framework that uses ChatGPT as a task planner to select and orchestrate different models from the HuggingFace library to solve AI tasks. It comprises four stages: task planning by ChatGPT, model selection, model execution, and result aggregation. ChatGPT parses the user request into multiple tasks with associated attributes like task type, ID, dependencies, and arguments.
